In [125]:
import pandas as pd
import examen
import examen.cargar_datos as cd
import examen.validar_datos as vd
import examen.perfil_datos as ped
import examen.analiticas as an
from IPython.display import display
from importlib import reload

pd.set_option('display.max_colwidth', None)

cd, vd, ped, an = map(reload, (cd, vd, ped, an))

## Extracción

Se crearon dos módulos dentro del paquete `examen`:
- `cargar_datos.py`: carga de tablas desde el archivo .zip ubicado en `data/` (sin extraer a disco).
- `validar_datos.py`: validación simple comparando pares CSV vs XLSX.

Tras generar reportes completos de diferencias (NaN-safe) para los pares disponibles, se decidió
utilizar los archivos CSV como fuente principal para uniones (joins) y pasos posteriores,
manteniendo los XLSX como referencia.

In [126]:
dfs = cd.cargar_dfs_desde_zip_en_data()
list(dfs.keys())

Leyendo datos/parking.csv -> key=parking_csv
Leyendo datos/usercuisine.csv -> key=usercuisine_csv
Leyendo datos/restaurants.xlsx -> key=restaurants_xlsx
Leyendo datos/users.xlsx -> key=users_xlsx
Leyendo datos/restaurants.csv -> key=restaurants_csv
Leyendo datos/cuisine.csv -> key=cuisine_csv
Leyendo datos/users.csv -> key=users_csv
Leyendo datos/ratings.csv -> key=ratings_csv
Leyendo datos/hours.csv -> key=hours_csv
Leyendo datos/userpayment.csv -> key=userpayment_csv
Leyendo datos/payment_methods.csv -> key=payment_methods_csv


['parking_csv',
 'usercuisine_csv',
 'restaurants_xlsx',
 'users_xlsx',
 'restaurants_csv',
 'cuisine_csv',
 'users_csv',
 'ratings_csv',
 'hours_csv',
 'userpayment_csv',
 'payment_methods_csv']

In [127]:
reps = vd.reporte_todos_pares_completo(dfs)

for base, rep in reps.items():
    print(rep["summary"])
    display(rep["diffs"])

{'base': 'restaurants', 'csv_shape': (130, 21), 'xlsx_shape': (130, 21), 'same_columns': True, 'equals_pandas': False, 'mismatch_cells': 107, 'note': ''}


,row,col,csv,xlsx
0,1,zip,78280,78280
1,2,latitude,22.149709,22.149709
2,2,longitude,-100.976093,-100.976093
3,2,zip,78000,78000
4,3,city,victoria,victoria
...,...,...,...,...
102,126,latitude,22.149192,22.149192
103,126,zip,78220,78220
104,128,latitude,18.875011,18.875011
105,128,longitude,-99.159422,-99.159422


{'base': 'users', 'csv_shape': (138, 19), 'xlsx_shape': (138, 19), 'same_columns': True, 'equals_pandas': False, 'mismatch_cells': 173, 'note': ''}


,row,col,csv,xlsx
0,0,smoker,false,False
1,1,smoker,false,False
2,2,smoker,false,False
3,3,longitude,-99.183,-99.183
4,3,smoker,false,False
...,...,...,...,...
168,134,smoker,false,False
169,135,smoker,true,True
170,136,longitude,-100.944623,-100.944623
171,136,smoker,false,False


In [128]:
dfs_csv = cd.obtener_csvs(dfs)

## Perfilado de DataFrames (CSV)

En esta sección se realiza el **perfilado exploratorio** de todos los DataFrames provenientes de archivos **CSV**, con el objetivo de:
- identificar estructura (filas/columnas),
- revisar distribución y tipo de variables,
- detectar valores faltantes y posibles inconsistencias,
- y documentar la calidad inicial de los datos antes de la integración/modelado.

In [129]:
dfs_perfil = ped.perfilar_todos_los_dfs(dfs_csv)

perfil_all = pd.concat(
    [p.assign(df=name) for name, p in dfs_perfil.items()],
    ignore_index=True
)

cols = ["df","Variable","Tipo_inferido","Dtype","Nulos","%Nulos","Unicos","%Cardinalidad","Min","Max","Media","Moda","Top"]
perfil_all = perfil_all[[c for c in cols if c in perfil_all.columns]]

perfil_all


/var/folders/16/1zxk1z8j1qz87x958_rnc9f80000gn/T/ipykernel_52177/11677862.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  perfil_all = pd.concat(


,df,Variable,Tipo_inferido,Dtype,Nulos,%Nulos,Unicos,%Cardinalidad,Min,Max,Media,Moda,Top
0,users_csv,userID,Texto,object,0,0.0,138,100.00,NaN,NaN,NaN,U1001,"U1001 (1), U1095 (1), U1089 (1), U1090 (1), U1091 (1), U1092 (1), U1093 (1), U1094 (1), U1096 (1), U1104 (1), Otros (128)"
1,users_csv,latitude,Numerica,float64,0,0.0,128,92.75,18.813348,23.771030,21.810389,None,None
2,users_csv,longitude,Numerica,float64,0,0.0,126,91.30,-101.054680,-99.067106,-100.291857,None,None
3,users_csv,weight,Numerica,int64,0,0.0,49,35.51,40.000000,120.000000,64.869565,None,None
4,users_csv,height,Numerica,float64,0,0.0,38,27.54,1.200000,2.000000,1.667536,None,None
5,users_csv,birth_year,Numerica,int64,0,0.0,21,15.22,1930.000000,1994.000000,1984.702899,None,None
6,users_csv,color,Categorica,object,0,0.0,8,5.80,NaN,NaN,NaN,blue,"blue (45), black (21), green (19), red (15), yellow (12), purple (11), white (11), orange (4)"
7,users_csv,dress_preference,Categorica,object,0,0.0,5,3.62,NaN,NaN,NaN,no preference,"no preference (53), formal (41), informal (35), ? (5), elegant (4)"
8,users_csv,interest,Categorica,object,0,0.0,5,3.62,NaN,NaN,NaN,variety,"variety (50), technology (36), none (30), eco-friendly (16), retro (6)"
9,users_csv,religion,Categorica,object,0,0.0,5,3.62,NaN,NaN,NaN,Catholic,"Catholic (99), none (30), Christian (7), Mormon (1), Jewish (1)"


In [130]:
ped.reporte_relacional(dfs_csv, ratings_key="ratings_csv")

,df,key,rows,unique_keys,min_per_key,max_per_key,mean_per_key,keys_in_universe,pct_keys_in_universe
0,users_csv,userID,138,138,1,1,1.000000,138,1.000000
1,usercuisine_csv,userID,330,138,1,103,2.391304,138,1.000000
2,userpayment_csv,userID,177,133,1,4,1.330827,133,1.000000
3,ratings_csv,userID,1161,138,3,18,8.413043,138,1.000000
4,ratings_csv,placeID,1161,130,3,36,8.930769,130,1.000000
5,restaurants_csv,placeID,130,130,1,1,1.000000,130,1.000000
6,parking_csv,placeID,702,675,1,3,1.040000,130,0.192593
7,cuisine_csv,placeID,916,769,1,9,1.191157,95,0.123537
8,payment_methods_csv,placeID,1314,615,1,8,2.136585,114,0.185366
9,hours_csv,placeID,2339,694,2,12,3.370317,128,0.184438


## Unidad muestral y modelo relacional

A partir del **reporte relacional** (`reporte_relacional`) se identificó que la tabla `ratings_csv` funciona como **tabla de hechos**, ya que contiene directamente las variables de calificación (`rating`, `food_rating`, `service_rating`) y conecta a usuarios y restaurantes mediante las llaves `userID` y `placeID`.

### Evidencia y justificación

- **Tabla de hechos:** `ratings_csv` (1161 filas).
- **Llaves presentes:** `userID` (138 únicos) y `placeID` (130 únicos).
- **Unidad muestral definida:** **1 fila = 1 evaluación** de un usuario (`userID`) a un restaurante (`placeID`).
- **Llave lógica de la evaluación:** (`userID`, `placeID`), y se verificó que **no existen duplicados** para este par, por lo que cada registro representa una evaluación única.

En consecuencia, la base principal para análisis y modelado se construye **partiendo de `ratings_csv`**, preservando su número de filas y, por lo tanto, su unidad muestral.

### Dimensiones analíticas para enriquecer la base

Con el mismo reporte relacional se observó que:
- `users_csv` es **1 a 1** con `userID` (mean_per_key = 1), por lo que puede integrarse directamente a nivel usuario.
- `restaurants_csv` es **1 a 1** con `placeID` (mean_per_key = 1), por lo que puede integrarse directamente a nivel restaurante.
- Otras tablas (`usercuisine_csv`, `userpayment_csv`, `cuisine_csv`, `payment_methods_csv`, `hours_csv`, `parking_csv`) presentan relaciones **1 a muchos** respecto a `userID` o `placeID`, por lo que se transforman previamente mediante **agregaciones** (conteos y listas de categorías) para mantener una sola fila por llave.

Para mantener el notebook legible y robustecer la paquetería, se implementaron funciones que construyen:
- `dim_user` (**usuarios.csv**): tabla analítica a nivel `userID` (1 fila por usuario), integrando `users_csv` y agregados por `userID`.
- `dim_place` (**restaurantes.csv**): tabla analítica a nivel `placeID` (1 fila por restaurante), integrando `restaurants_csv` y agregados por `placeID`.

Finalmente, la **base de modelado** se obtiene realizando `LEFT JOIN` desde `ratings_csv` hacia `dim_user` y `dim_place`, asegurando que el número de filas permanezca en 1161 y que la unidad muestral continúe siendo **la evaluación usuario–restaurante**.


In [131]:
dim_user = an.construir_dim_usuario(dfs_csv)
dim_place = an.construir_dim_restaurante(dfs_csv)
an.exportar_tablas_analiticas(dim_user, dim_place, out_dir="./data")

base = an.construir_base_modelado(dfs_csv, dim_user, dim_place)

## Ingeniería de datos: variables nuevas a nivel de la unidad muestral (`ratings`)

Con la unidad muestral definida como **1 fila = 1 evaluación** (`userID`, `placeID`), se construyeron variables adicionales **derivadas directamente de las calificaciones** de `ratings_csv`. Estas variables se calculan al mismo grano de la unidad muestral (no generan explosión de filas) y buscan capturar:

- La **intensidad promedio** de la evaluación (señal global).
- La **consistencia o discrepancia** entre componentes (comida vs servicio).
- El **contexto del evaluador** (tendencia típica del usuario) y del evaluado (tendencia típica del restaurante), calculados con enfoque *leave-one-out* para no usar la misma observación en su propio promedio.

### Variables creadas (≥ 5) y justificación

- **`subrating_mean`**: resume en una sola señal el nivel promedio entre `food_rating` y `service_rating`, útil para modelos que aprovechan una medida global de calidad.
- **`subrating_gap`**: captura si la evaluación favorece más la comida o el servicio; ayuda a distinguir perfiles donde una dimensión domina a la otra.
- **`subrating_gap_abs`**: mide la consistencia entre comida y servicio; discrepancias grandes pueden indicar experiencias “mixtas” que un promedio no detecta.
- **`user_mean_rating_loo`**: representa la tendencia típica del usuario a calificar alto/bajo (sesgo del evaluador) sin incluir la evaluación actual; permite ajustar por usuarios más exigentes o más complacientes.
- **`place_mean_rating_loo`**: representa la tendencia típica del restaurante a recibir mejores/peores calificaciones sin incluir la evaluación actual; provee un contexto de reputación promedio a nivel `placeID`.

In [132]:
base = an.agregar_features_ratings(base)

cols_new = [
    "subrating_mean",
    "subrating_gap",
    "subrating_gap_abs",
    "user_mean_rating_loo",
    "place_mean_rating_loo",
]
base


,userID,placeID,rating,food_rating,service_rating,latitude,longitude,smoker,drink_level,dress_preference,...,hours_Wed,hours_Thu,hours_Fri,hours_Sat,hours_Sun,subrating_mean,subrating_gap,subrating_gap_abs,user_mean_rating_loo,place_mean_rating_loo
0,U1077,135085,2,2,2,22.156469,-100.985540,False,social drinker,elegant,...,00:00-00:00,00:00-00:00,00:00-00:00,00:00-00:00,00:00-00:00,2.0,0,0,1.250000,1.314286
1,U1077,135038,2,2,1,22.156469,-100.985540,False,social drinker,elegant,...,08:00-17:00,08:00-17:00,08:00-17:00,08:00-17:00,00:00-00:00,1.5,1,1,1.250000,1.173913
2,U1077,132825,2,2,2,22.156469,-100.985540,False,social drinker,elegant,...,09:00-12:00,09:00-12:00,09:00-12:00,09:00-12:00,09:00-12:00,2.0,0,0,1.250000,1.258065
3,U1077,135060,1,2,2,22.156469,-100.985540,False,social drinker,elegant,...,11:30-19:00,11:30-19:00,11:30-19:00,11:30-19:00,11:30-19:00,2.0,0,0,1.500000,1.142857
4,U1068,135104,1,1,2,23.752269,-99.168605,False,casual drinker,informal,...,00:00-23:30,00:00-23:30,00:00-23:30,00:00-23:30,00:00-23:30,1.5,-1,1,0.571429,0.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1156,U1043,132630,1,1,1,23.771030,-99.167082,False,abstemious,no preference,...,00:00-23:30,00:00-23:30,00:00-23:30,00:00-23:30,00:00-23:30,1.0,0,0,1.000000,1.200000
1157,U1011,132715,1,1,0,23.724972,-99.152856,False,abstemious,no preference,...,09:00-16:00,09:00-16:00,09:00-16:00,09:00-16:00,09:00-16:00,0.5,1,1,1.500000,1.000000
1158,U1068,132733,1,1,0,23.752269,-99.168605,False,casual drinker,informal,...,10:30-22:30,10:30-22:30,10:30-22:30,10:30-22:30,09:00-21:00,0.5,1,1,0.571429,1.333333
1159,U1068,132594,1,1,1,23.752269,-99.168605,False,casual drinker,informal,...,09:30-23:30,09:30-23:30,09:30-23:30,09:00-23:30,09:00-23:30,1.0,0,0,0.571429,0.500000


In [133]:
base.shape[1]

66

In [134]:
usercuisine = dfs_csv["usercuisine_csv"] 
users = dfs_csv["users_csv"] 
userpayment = dfs_csv["userpayment_csv"] 
ratings = dfs_csv["ratings_csv"] 
parking = dfs_csv["parking_csv"] 
restaurants = dfs_csv["restaurants_csv"] 
cuisine = dfs_csv["cuisine_csv"] 
payment_methods = dfs_csv["payment_methods_csv"] 
hours = dfs_csv["hours_csv"]

In [135]:
base

,userID,placeID,rating,food_rating,service_rating,latitude,longitude,smoker,drink_level,dress_preference,...,hours_Wed,hours_Thu,hours_Fri,hours_Sat,hours_Sun,subrating_mean,subrating_gap,subrating_gap_abs,user_mean_rating_loo,place_mean_rating_loo
0,U1077,135085,2,2,2,22.156469,-100.985540,False,social drinker,elegant,...,00:00-00:00,00:00-00:00,00:00-00:00,00:00-00:00,00:00-00:00,2.0,0,0,1.250000,1.314286
1,U1077,135038,2,2,1,22.156469,-100.985540,False,social drinker,elegant,...,08:00-17:00,08:00-17:00,08:00-17:00,08:00-17:00,00:00-00:00,1.5,1,1,1.250000,1.173913
2,U1077,132825,2,2,2,22.156469,-100.985540,False,social drinker,elegant,...,09:00-12:00,09:00-12:00,09:00-12:00,09:00-12:00,09:00-12:00,2.0,0,0,1.250000,1.258065
3,U1077,135060,1,2,2,22.156469,-100.985540,False,social drinker,elegant,...,11:30-19:00,11:30-19:00,11:30-19:00,11:30-19:00,11:30-19:00,2.0,0,0,1.500000,1.142857
4,U1068,135104,1,1,2,23.752269,-99.168605,False,casual drinker,informal,...,00:00-23:30,00:00-23:30,00:00-23:30,00:00-23:30,00:00-23:30,1.5,-1,1,0.571429,0.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1156,U1043,132630,1,1,1,23.771030,-99.167082,False,abstemious,no preference,...,00:00-23:30,00:00-23:30,00:00-23:30,00:00-23:30,00:00-23:30,1.0,0,0,1.000000,1.200000
1157,U1011,132715,1,1,0,23.724972,-99.152856,False,abstemious,no preference,...,09:00-16:00,09:00-16:00,09:00-16:00,09:00-16:00,09:00-16:00,0.5,1,1,1.500000,1.000000
1158,U1068,132733,1,1,0,23.752269,-99.168605,False,casual drinker,informal,...,10:30-22:30,10:30-22:30,10:30-22:30,10:30-22:30,09:00-21:00,0.5,1,1,0.571429,1.333333
1159,U1068,132594,1,1,1,23.752269,-99.168605,False,casual drinker,informal,...,09:30-23:30,09:30-23:30,09:30-23:30,09:00-23:30,09:00-23:30,1.0,0,0,0.571429,0.500000


In [136]:
base.to_csv("base.csv", index=False)